In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 92kB 7.9MB/s eta 0:00:011
Libraries imported.


### Set up Foursquare API

In [3]:
CLIENT_ID = 'WW0EUZCFGI1ZE2K0VMGJNNW5YBBNU1IJSXGBAZ2X2P1H3WTV' # your Foursquare ID
CLIENT_SECRET = '0NSKFI40ISUU2GEGY2IENFD3NCQWSECYGEFZLLXJTYXSUEVU' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WW0EUZCFGI1ZE2K0VMGJNNW5YBBNU1IJSXGBAZ2X2P1H3WTV
CLIENT_SECRET:0NSKFI40ISUU2GEGY2IENFD3NCQWSECYGEFZLLXJTYXSUEVU


### Use geopy library to get the latitude and longitude values of Singapore.

In [20]:
address = 'Singapore,Singapore '

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Singapore are 1.357107, 103.8194992.


### Create a map of Singapore

In [8]:
# create map of New York using latitude and longitude values
map_singapore = folium.Map(location=[latitude, longitude], zoom_start=10)
    
map_singapore

### I want to find which neighborhood has the most hotels and tourist attractions.

In [83]:
# Store the data into a panda dataframe
url = 'https://en.wikipedia.org/wiki/Regions_of_Singapore'
list_df = pd.read_html(url, header = 0)
region = list_df[1]
region = region[["Region[3]","Area(km²)",'Estimated Population[4]']]
region = region.drop(5)

region.insert(3,"Latitude",[1.2789,1.3496,1.4382,1.4052,1.3329])
region.insert(4,"Longitude",[103.8536,103.9568,103.7890,103.8663,103.7436])
region

,Region[3],Area(km²),Estimated Population[4],Latitude,Longitude
0,Central Region,132.7,922980,1.2789,103.8536
1,East Region,93.1,686050,1.3496,103.9568
2,North Region,134.5,573950,1.4382,103.7890
3,North-East Region,103.9,921940,1.4052,103.8663
4,West Region,201.3,921340,1.3329,103.7436


In [86]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Region', 
                  'Region Latitude', 
                  'Region Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [1]:
singapore_venues = getNearbyVenues(names=region["Region[3]"],
                                   latitudes=region["Latitude"],
                                   longitudes=region['Longitude']
                                  )

singapore_venues.head()

NameError: name 'getNearbyVenues' is not defined

In [96]:
# one hot encoding
singapore_onehot = pd.get_dummies(singapore_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
singapore_onehot['Region'] = singapore_venues['Region'] 

# move neighborhood column to the first column
fixed_columns = [singapore_onehot.columns[-1]] + list(singapore_onehot.columns[:-1])
singapore_onehot = singapore_onehot[fixed_columns]

singapore_grouped = singapore_onehot.groupby('Region').mean().reset_index()
singapore_grouped

num_top_venues = 10
for hood in singapore_grouped['Region']:
    print("----"+hood+"----")
    temp = singapore_grouped[singapore_grouped['Region'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Region----
                  venue  freq
0                 Hotel  0.12
1    Italian Restaurant  0.03
2           Coffee Shop  0.03
3            Restaurant  0.03
4  Gym / Fitness Center  0.03
5            Waterfront  0.03
6                  Café  0.03
7   Japanese Restaurant  0.02
8                 Plaza  0.02
9     Korean Restaurant  0.02


----East Region----
                  venue  freq
0           Coffee Shop  0.13
1                  Café  0.06
2                Bakery  0.05
3   Japanese Restaurant  0.05
4            Food Court  0.04
5  Fast Food Restaurant  0.04
6        Sandwich Place  0.04
7                   Gym  0.03
8           Supermarket  0.03
9          Dessert Shop  0.03


----North Region----
                  venue  freq
0            Food Court  0.16
1           Coffee Shop  0.06
2  Fast Food Restaurant  0.05
3      Asian Restaurant  0.05
4                  Park  0.05
5         Grocery Store  0.03
6    Chinese Restaurant  0.03
7                  Café  0.03
8 

In [162]:
# put these into a panda dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Region']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Region'] = singapore_grouped['Region']

for ind in np.arange(singapore_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(singapore_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Region,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Region,Hotel,Italian Restaurant,Waterfront,Coffee Shop,Café,Restaurant,Gym / Fitness Center,Sandwich Place,Cocktail Bar,Korean Restaurant
1,East Region,Coffee Shop,Café,Bakery,Japanese Restaurant,Sandwich Place,Fast Food Restaurant,Food Court,Shopping Mall,Dessert Shop,Supermarket
2,North Region,Food Court,Coffee Shop,Park,Fast Food Restaurant,Asian Restaurant,Grocery Store,Shopping Mall,Café,Chinese Restaurant,Supermarket
3,North-East Region,Café,Asian Restaurant,Coffee Shop,Airport,Fast Food Restaurant,Dessert Shop,Food Court,Thai Restaurant,Airport Terminal,Restaurant
4,West Region,Chinese Restaurant,Food Court,Coffee Shop,Japanese Restaurant,Café,Supermarket,Shopping Mall,Indian Restaurant,Garden,Fast Food Restaurant


#### It seems like the central region is the most ideal location to open a travel agency. I want to narrow down on the areas that most tourists are attracted to.

### Use geopy library to get the latitude and longitude values of Central Area, Singapore.

In [97]:
address = 'Downtown,Singapore '

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_dt = folium.Map(location=[latitude, longitude], zoom_start=15)
    
map_dt

In [105]:
# Store the data into a panda dataframe
url = 'https://en.wikipedia.org/wiki/Planning_Areas_of_Singapore'
list_df = pd.read_html(url, header = 0)
area = list_df[2]
area = area[area['Region']=='Central'].reset_index(drop=True)
area = area[["Name (English)","Area (km2)","Population[7]"]]
area

area_lat = [1.3526,1.2819,1.3294,1.2867,1.3201,1.3100,1.2913,1.2793,1.3020,
            1.2966,1.3076,1.3209,1.3048,1.2849,1.2942,1.2959,1.3051,1.2906,
            1.2537,1.2729,1.3069,1.3343]
area_long = [103.8352,103.8239,103.8021,103.8535,103.8918,103.8651,103.8709,
             103.8702,103.8971,103.8485,103.8404,103.8424,103.8318,103.8439,
             103.7861,103.8361,103.8509,103.8465,103.8257,103.8627,103.8189,
             103.8563]

area.insert(3,"Latitude",area_lat)
area.insert(4,"Longitude",area_long)
area

,Name (English),Area (km2),Population[7],Latitude,Longitude
0,Bishan,7.62,88010,1.3526,103.8352
1,Bukit Merah,14.34,151980,1.2819,103.8239
2,Bukit Timah,17.53,77430,1.3294,103.8021
3,Downtown Core,4.34,2720,1.2867,103.8535
4,Geylang,9.64,110200,1.3201,103.8918
5,Kallang,9.17,101520,1.3100,103.8651
6,Marina East,1.82,*,1.2913,103.8709
7,Marina South,1.62,*,1.2793,103.8702
8,Marine Parade,6.12,46390,1.3020,103.8971
9,Museum,0.83,420,1.2966,103.8485


In [107]:
central_venues = getNearbyVenues(names=area["Name (English)"],
                                   latitudes=area["Latitude"],
                                   longitudes=area['Longitude']
                                  )

central_venues

Bishan
Bukit Merah
Bukit Timah
Downtown Core
Geylang
Kallang
Marina East
Marina South
Marine Parade
Museum
Newton
Novena
Orchard
Outram
Queenstown
River Valley
Rochor
Singapore River
Southern Islands
Straits View
Tanglin
Toa Payoh


,Region,Region Latitude,Region Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bishan,1.3526,103.8352,Chocolat N' Spice,1.351867,103.837558,Bakery
1,Bishan,1.3526,103.8352,Tomyum Mama,1.353415,103.834441,Thai Restaurant
2,Bishan,1.3526,103.8352,Sin Ming Roti Prata (Faisal & Aziz Curry Musli...,1.355269,103.836719,Indian Restaurant
3,Bishan,1.3526,103.8352,Columbus Coffee Co.,1.352790,103.835391,Café
4,Bishan,1.3526,103.8352,Brownice,1.353639,103.835868,Ice Cream Shop
5,Bishan,1.3526,103.8352,MacRitchie Nature Trail,1.347312,103.831411,Trail
6,Bishan,1.3526,103.8352,Healing Touch,1.351915,103.835479,Spa
7,Bishan,1.3526,103.8352,Anti-Oxidant,1.353384,103.834082,Spa
8,Bishan,1.3526,103.8352,One Man Coffee,1.352875,103.834766,Café
9,Bishan,1.3526,103.8352,MacRitchie Reservoir,1.342698,103.833632,Reservoir


In [118]:
central_venues.groupby('Region').count()

# one hot encoding
central_onehot = pd.get_dummies(central_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
central_onehot['Region'] = central_venues['Region'] 

# move neighborhood column to the first column
fixed_columns = [central_onehot.columns[-1]] + list(central_onehot.columns[:-1])
central_onehot = central_onehot[fixed_columns]

central_onehot.head()

,Region,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bay,Beach,Beer Bar,Beer Garden,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burrito Place,Business Service,Butcher,Café,Candy Store,Cantonese Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Discount Store,Dog Run,Dumpling Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flower Shop,Food,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Fujian Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Harbor / Marina,History Museum,Hobby Shop,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Library,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Mountain,Movie Theater,Multiplex,Museum,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Paella Restaurant,Park,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Pet Service,Pet Store,Pie Shop,Pizza Place,Playground,Plaza,Pool,Pub,Racetrack,Ramen Restaurant,Recreation Center,Reservoir,Resort,Restaurant,River,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shaanxi Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Club,Stables,Stadium,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Swiss Restaurant,TV Station,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Tourist Information Center,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio
0,Bishan,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Bishan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [119]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
central_grouped = central_onehot.groupby('Region').mean().reset_index()
central_grouped

num_top_venues = 5

# print each neighborhood along with the top 5 most common venues
for hood in central_grouped['Region']:
    print("----"+hood+"----")
    temp = central_grouped[central_grouped['Region'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bishan----
                venue  freq
0  Chinese Restaurant  0.12
1                Café  0.06
2         Coffee Shop  0.06
3         Flower Shop  0.06
4      Ice Cream Shop  0.06


----Bukit Merah----
                 venue  freq
0  Japanese Restaurant  0.08
1                 Café  0.08
2                Hotel  0.06
3           Food Court  0.05
4                 Park  0.05


----Bukit Timah----
                 venue  freq
0               Bakery  0.07
1                 Café  0.06
2  Japanese Restaurant  0.06
3   Italian Restaurant  0.05
4   Chinese Restaurant  0.05


----Downtown Core----
                venue  freq
0               Hotel  0.15
1          Waterfront  0.05
2       Shopping Mall  0.03
3  Italian Restaurant  0.02
4          Restaurant  0.02


----Geylang----
                venue  freq
0  Chinese Restaurant  0.11
1    Asian Restaurant  0.08
2          Food Court  0.07
3  Seafood Restaurant  0.06
4        Noodle House  0.04


----Kallang----
                venue  freq
0

In [120]:
# put these into a panda dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Region']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Region'] = central_grouped['Region']

for ind in np.arange(central_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(central_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Region,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bishan,Chinese Restaurant,Coffee Shop,Ice Cream Shop,Flower Shop,Café,Food Court,Thai Restaurant,Spa,Park,Seafood Restaurant
1,Bukit Merah,Café,Japanese Restaurant,Hotel,Food Court,Park,Coffee Shop,Scenic Lookout,Chinese Restaurant,Bookstore,Supermarket
2,Bukit Timah,Bakery,Café,Japanese Restaurant,Chinese Restaurant,Italian Restaurant,Soccer Field,Coffee Shop,Pizza Place,Indian Restaurant,Thai Restaurant
3,Downtown Core,Hotel,Waterfront,Shopping Mall,Gym,Italian Restaurant,Event Space,Boutique,Coffee Shop,Performing Arts Venue,Cocktail Bar
4,Geylang,Chinese Restaurant,Asian Restaurant,Food Court,Seafood Restaurant,Noodle House,BBQ Joint,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Dim Sum Restaurant,Italian Restaurant


### Cluster the planning areas

In [150]:
# Run *k*-means to cluster the neighborhood into 5 clusters
# set number of clusters
kclusters = 5

central_grouped_clustering = central_grouped.drop('Region', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(central_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 3, 1, 0, 1, 3, 4, 4, 1, 0], dtype=int32)

In [151]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
# add clustering labels

central_merged = area

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
central_merged = central_merged.join(neighborhoods_venues_sorted.set_index('Region'), on='Name (English)')
central_merged.head()

,Name (English),Area (km2),Population[7],Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bishan,7.62,88010,1.3526,103.8352,1,Chinese Restaurant,Coffee Shop,Ice Cream Shop,Flower Shop,Café,Food Court,Thai Restaurant,Spa,Park,Seafood Restaurant
1,Bukit Merah,14.34,151980,1.2819,103.8239,3,Café,Japanese Restaurant,Hotel,Food Court,Park,Coffee Shop,Scenic Lookout,Chinese Restaurant,Bookstore,Supermarket
2,Bukit Timah,17.53,77430,1.3294,103.8021,1,Bakery,Café,Japanese Restaurant,Chinese Restaurant,Italian Restaurant,Soccer Field,Coffee Shop,Pizza Place,Indian Restaurant,Thai Restaurant
3,Downtown Core,4.34,2720,1.2867,103.8535,0,Hotel,Waterfront,Shopping Mall,Gym,Italian Restaurant,Event Space,Boutique,Coffee Shop,Performing Arts Venue,Cocktail Bar
4,Geylang,9.64,110200,1.3201,103.8918,1,Chinese Restaurant,Asian Restaurant,Food Court,Seafood Restaurant,Noodle House,BBQ Joint,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Dim Sum Restaurant,Italian Restaurant


### Visualize the clusters

In [163]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(central_merged['Latitude'], central_merged['Longitude'], central_merged['Name (English)'], central_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine the clusters

In [153]:
central_merged.loc[central_merged['Cluster Labels'] == 0, central_merged.columns[[0] + list(range(5, central_merged.shape[1]))]]

,Name (English),Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Downtown Core,0,Hotel,Waterfront,Shopping Mall,Gym,Italian Restaurant,Event Space,Boutique,Coffee Shop,Performing Arts Venue,Cocktail Bar
9,Museum,0,Hotel,Japanese Restaurant,Wine Bar,Cocktail Bar,Ice Cream Shop,Bistro,Movie Theater,Park,Performing Arts Venue,Shopping Mall
13,Outram,0,Hotel,Japanese Restaurant,Wine Bar,Coffee Shop,Yoga Studio,Spanish Restaurant,Korean Restaurant,Café,Cocktail Bar,Gym
17,Singapore River,0,Hotel,Japanese Restaurant,Wine Bar,Cocktail Bar,French Restaurant,Waterfront,Plaza,Beer Garden,Café,Park


In [154]:
central_merged.loc[central_merged['Cluster Labels'] == 1, central_merged.columns[[0] + list(range(5, central_merged.shape[1]))]]

,Name (English),Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bishan,1,Chinese Restaurant,Coffee Shop,Ice Cream Shop,Flower Shop,Café,Food Court,Thai Restaurant,Spa,Park,Seafood Restaurant
2,Bukit Timah,1,Bakery,Café,Japanese Restaurant,Chinese Restaurant,Italian Restaurant,Soccer Field,Coffee Shop,Pizza Place,Indian Restaurant,Thai Restaurant
4,Geylang,1,Chinese Restaurant,Asian Restaurant,Food Court,Seafood Restaurant,Noodle House,BBQ Joint,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Dim Sum Restaurant,Italian Restaurant
8,Marine Parade,1,Chinese Restaurant,Hotel,Coffee Shop,Bar,Multiplex,Seafood Restaurant,Vietnamese Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant
14,Queenstown,1,Chinese Restaurant,Coffee Shop,Food Court,Sandwich Place,Japanese Restaurant,Indian Restaurant,Gym / Fitness Center,Café,Italian Restaurant,Supermarket
21,Toa Payoh,1,Chinese Restaurant,Food Court,Bakery,Noodle House,Coffee Shop,Dessert Shop,Seafood Restaurant,Asian Restaurant,Hotel,Supermarket


In [155]:
central_merged.loc[central_merged['Cluster Labels'] == 2, central_merged.columns[[0] + list(range(5, central_merged.shape[1]))]]

,Name (English),Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Southern Islands,2,Theme Park Ride / Attraction,Theme Park,Beach,Scenic Lookout,Restaurant,Spa,Hotel,Resort,Shoe Store,Chocolate Shop


In [156]:
central_merged.loc[central_merged['Cluster Labels'] == 3, central_merged.columns[[0] + list(range(5, central_merged.shape[1]))]]

,Name (English),Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Bukit Merah,3,Café,Japanese Restaurant,Hotel,Food Court,Park,Coffee Shop,Scenic Lookout,Chinese Restaurant,Bookstore,Supermarket
5,Kallang,3,Hotel,Chinese Restaurant,Indian Restaurant,Thai Restaurant,Bakery,BBQ Joint,Coffee Shop,Food Court,Supermarket,Restaurant
10,Newton,3,Hotel,Japanese Restaurant,Shopping Mall,Sushi Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Boutique,Bubble Tea Shop
11,Novena,3,Hotel,Indian Restaurant,Chinese Restaurant,Café,Bakery,Restaurant,Food Court,Japanese Restaurant,Italian Restaurant,Dessert Shop
12,Orchard,3,Hotel,Japanese Restaurant,Shopping Mall,Boutique,Sushi Restaurant,Bakery,Coffee Shop,Clothing Store,Chinese Restaurant,Cosmetics Shop
15,River Valley,3,Hotel,Japanese Restaurant,Shopping Mall,Sushi Restaurant,Coffee Shop,Wine Bar,Bakery,Cosmetics Shop,Bubble Tea Shop,Chinese Restaurant
16,Rochor,3,Hotel,Café,Indian Restaurant,Chinese Restaurant,Japanese Restaurant,Clothing Store,Ice Cream Shop,Vegetarian / Vegan Restaurant,Bakery,Cocktail Bar
20,Tanglin,3,Hotel,Japanese Restaurant,Sushi Restaurant,Boutique,Garden,French Restaurant,Chinese Restaurant,Coffee Shop,Cosmetics Shop,Modern European Restaurant


In [157]:
central_merged.loc[central_merged['Cluster Labels'] == 4, central_merged.columns[[0] + list(range(5, central_merged.shape[1]))]]

,Name (English),Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Marina East,4,Hotel,Scenic Lookout,Garden,Bridge,Italian Restaurant,Thai Restaurant,Hotel Bar,Event Space,American Restaurant,Hotel Pool
7,Marina South,4,Hotel,Scenic Lookout,Waterfront,Garden,Italian Restaurant,Hotel Bar,Bridge,Café,Japanese Restaurant,Buffet
19,Straits View,4,Hotel,Scenic Lookout,Garden,Bridge,Waterfront,Italian Restaurant,Gym / Fitness Center,Gastropub,Japanese Restaurant,Lounge
